In [14]:
## Imports y definiciones
import numpy as np
import pickle
import warnings
from sklearn.datasets.base import Bunch
from sklearn.model_selection import GridSearchCV, StratifiedKFold


## Clasificadores
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import neural_network
from sklearn.naive_bayes import MultinomialNB



def load_my_dataset():
    with open("/home/dpedro/bow_twitter.pkl", 'rb') as pkl_file:
        data_file = pickle.load(pkl_file)
        n_samples = len(data_file) #501 #150 #number of data rows, don't count header
        n_features = len([x.name for x in data_file.domain.attributes]) #number of feature columns, don't count target
        feature_names = [x.name for x in data_file.domain.attributes] #adjust accordingly
        target_names = [x for x in data_file.domain.class_var.values] #adjust accordingly
        data = np.empty((n_samples, n_features))
        target = np.empty((n_samples,), dtype=np.int)

        for i in range(len(data)):
            data[i] = np.asarray(data_file[i,0:2222], dtype=np.float64)
            target[i] = np.asarray(data_file[i,2222], dtype=np.int)

    return Bunch(data=data, target=target, feature_names = feature_names, target_names = target_names)


warnings.filterwarnings('ignore')



data = load_my_dataset()    # Cargamos el dataset

## Main
if __name__ == "__main__":
    
    # --- Parameters ---
    svm_parameters = [{ 'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5], 'C': [1, 10, 100, 1000]},
                      { 'kernel': ['linear'], 'C': [1, 10, 100, 1000] }]
    
    tree_parameters = [{'criterion': ['gini', 'entropy'],
                        'max_depth': [ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
                        'min_samples_leaf': [1, 5, 10, 20, 50, 100] }]
    
    nnet_parameters = [{'solver': ['lbfgs'], 
                        'max_iter': [500,1000,1500], 
                        'alpha': 10.0 ** -np.arange(1, 7), 
                        'hidden_layer_sizes':np.arange(5, 12), 
                        'random_state':[0,1,2,3,4,5,6,7,8,9] }]
    
    bay_parameters = [{ 'alpha': np.linspace(0.5, 1.5, 6),
                        'fit_prior': [True, False] }] 
    
    # --- Scores ---
    scores = ['precision', 'recall'] #'accuracy' por ahora no
    
    # --- Cros Validate ---
    cvNum = 10
    
    for score in scores:
        
        print("Buscando parametros de %s" % score)
        
        svm_clf = GridSearchCV( SVC(), svm_parameters, cv=cvNum, scoring='%s_weighted' % score, n_jobs=-1, verbose=1, refit=False)
        svm_clf.fit(data.data, data.target)
        print("Metrica: ", score)
        print("La mejor puntuacion para SVM fue: %0.3f" % svm_clf.best_score_)
        print("con los siguientes parametros: ")
        print(svm_clf.best_params_)
        print()

        treeclf = GridSearchCV(DecisionTreeClassifier(), tree_parameters, cv=cvNum, scoring='%s_weighted' % score, n_jobs=-1)
        treeclf.fit(data.data, data.target)
        print("Metrica: ", score)
        print("La mejor puntuacion para Random Forest fue: %0.3f" % treeclf.best_score_)
        print("con los siguientes parametros: ")
        print(treeclf.best_params_)
        print()
        
        nnetclf = GridSearchCV(neural_network.MLPClassifier(), nnet_parameters, cv=cvNum, scoring='%s_weighted' % score, n_jobs=-1)
        nnetclf.fit(data.data, data.target)
        print("Metrica: ", score)
        print("La mejor puntuacion para Red Neuronal fue: %0.3f" % nnetclf.best_score_)
        print("con los siguientes parametros: ")
        print(nnetclf.best_params_)
        print()
        
        bay_clf = GridSearchCV(MultinomialNB(), bay_parameters, cv=cvNum, scoring='%s_weighted' % score, n_jobs=-1)
        bay_clf.fit(data.data, data.target)
        print("Metrica: ", score)
        print("La mejor puntuacion para Naive Bayes fue: %0.3f" % bay_clf.best_score_)
        print("con los siguientes parametros: ")
        print(bay_clf.best_params_)
        print()
        

Buscando parametros de precision
Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.2min finished


Metrica:  precision
La mejor puntuacion para SVM fue: 0.665
con los siguientes parametros: 
{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Metrica:  precision
La mejor puntuacion para Random Forest fue: 0.649
con los siguientes parametros: 
{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 1}

Metrica:  precision
La mejor puntuacion para Red Neuronal fue: 0.691
con los siguientes parametros: 
{'alpha': 0.001, 'hidden_layer_sizes': 9, 'max_iter': 500, 'random_state': 9, 'solver': 'lbfgs'}

Metrica:  precision
La mejor puntuacion para Naive Bayes fue: 0.688
con los siguientes parametros: 
{'alpha': 1.5, 'fit_prior': True}

Buscando parametros de recall
Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.1min finished


Metrica:  recall
La mejor puntuacion para SVM fue: 0.659
con los siguientes parametros: 
{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Metrica:  recall
La mejor puntuacion para Random Forest fue: 0.611
con los siguientes parametros: 
{'criterion': 'gini', 'max_depth': 17, 'min_samples_leaf': 1}

Metrica:  recall
La mejor puntuacion para Red Neuronal fue: 0.685
con los siguientes parametros: 
{'alpha': 0.001, 'hidden_layer_sizes': 9, 'max_iter': 500, 'random_state': 0, 'solver': 'lbfgs'}

Metrica:  recall
La mejor puntuacion para Naive Bayes fue: 0.683
con los siguientes parametros: 
{'alpha': 1.5, 'fit_prior': True}

